# My Playground for Neural Network

In [59]:
from utils import *
from stratified import *
from run import *
from neuralnetwork import *

In [60]:
def importhousedata():
    house = importfile('hw3_house_votes_84.csv', ',')
    housecategory = {}
    for i in house[0]:
        housecategory[i] = 'categorical'
    housecategory["class"] = 'class'
    housedata = np.array(house[1:]).astype(float)
    return housedata, housecategory
def importwinedata():
    wine = importfile('hw3_wine.csv', '\t')
    winecategory = {}
    for i in wine[0]:
        winecategory[i] = 'numerical'
    winecategory["# class"] = 'class'
    winedata = np.array(wine[1:]).astype(float)
    return winedata, winecategory
def importcmcdata():
    cmc = importfile('cmc.data', ',')
    cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
    "Husband's education":"categorical","Number of children ever born":"numerical",
    "Wife's religion":"binary","Wife's now working?":"binary",
    "Husband's occupation":"categorical","Standard-of-living index":"categorical",
    "Media exposure":"binary","Contraceptive method used":"class"}
    cmcdata = np.array(cmc).astype(int)
    return cmcdata, cmccategory
def importcancerdata():
    cancer = importfile('hw3_cancer.csv', '\t')
    cancercategory = {}
    for i in cancer[0]:
        cancercategory[i] = 'numerical'
    cancercategory["Class"] = 'class'
    cancerdata = np.array(cancer[1:]).astype(float)
    return cancerdata, cancercategory


In [61]:
ohe_house_data, ohe_house_category = onehotencoder(housedata, housecategory)
ohe_wine_data, ohe_wine_category = onehotencoder(winedata,winecategory)
ohe_cmc_data, ohe_cmc_category = onehotencoder(cmcdata,cmccategory)
ohe_cancer_data, ohe_cancer_category = onehotencoder(cancerdata,cancercategory)

In [62]:
n_ohe_house_data = normalizetrain(ohe_house_data,ohe_house_category)[0]
n_ohe_wine_data = normalizetrain(ohe_wine_data,ohe_wine_category)[0]
n_ohe_cmc_data = normalizetrain(ohe_cmc_data,ohe_cmc_category)[0]
n_ohe_cancer_data = normalizetrain(ohe_cancer_data,ohe_cancer_category)[0]

In [63]:
def gradientD(weights_list,deltalist,attributelist,biasterm=True):
    gradlist = []
    for i in range(len(weights_list)):
        attributenow = attributelist[i]
        deltanow = np.array([deltalist[i]]).T
        dotproduct = deltanow*attributenow
        # print('dotshape',dotproduct.shape)
        gradlist.append(dotproduct)
    return gradlist

In [64]:
# Forward propagation vectorized
def neural_network(normed_hotted_data,ohe_category,weights_list, minibatchk = 15, lambda_reg = 0.15, learning_rate = 0.01):
    biasterm = True
    normed_ohe_copy = normed_hotted_data.copy()
    if minibatchk > len(normed_hotted_data):
        minibatchk = len(normed_hotted_data)
    np.random.shuffle(normed_ohe_copy)
    splitted = np.array_split(normed_ohe_copy, minibatchk)
    
    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm else 0
    
    for onebatch in splitted:
        onebatch = onebatch.T
        input_data = onebatch[inputindex].T
        output_data = onebatch[outputindex].T
        # input_data_mean = onebatch[inputindex].mean(axis=1)
        output_data = onebatch[outputindex].T

        # forward propagation
        instance_index = 0
        j = 0
        listofgradient = []
        for one_instance in input_data:
            current_layer_a = np.append(1,one_instance) if b == 1 else one_instance
            # input layer is the current layer
            current_layer_index = 0
            output_expect = output_data[instance_index]
            attributesnobias = [one_instance]
            attributeswbias = [current_layer_a]
            for theta in weights_list:
                z = np.dot(theta,current_layer_a)
                a = g(z)
                current_layer_a = np.append(1,a) if (b == 1) and (current_layer_index+1 != len(weights_list)) else a
                attributesnobias.append(a)
                attributeswbias.append(current_layer_a)
                current_layer_index += 1

            output_predict = current_layer_a # the last attribute is the output for this batch.
            instance_index += 1
            j += costfunction(output_expect,output_predict)

            # calculate delta blame (back propagation)
            listofdelta = blame(output_predict,output_expect,weights_list, attributeswbias)
            thisgradient = gradientD(weights_list,listofdelta,attributeswbias,biasterm)
            listofgradient.append(thisgradient)
        
        gradientP = [lambda_reg*t for t in weights_list]
        # first column in singleP in the np.array = 0
        for singleP in gradientP:
            singleP[:, 0] = 0
        
        grad_D_transpose = transposelistoflist(listofgradient)
        grad_D_sum = [np.sum(t,axis=0) for t in grad_D_transpose]
        gradients_batch = []
        for i in range(len(grad_D_sum)):
            gradients_batch.append((grad_D_sum[i] + gradientP[i])*(1/instance_index))
        
        j /= (instance_index+1)
        s = sumofweights(weights_list,bias=b)*lambda_reg/(2*(instance_index+1))
        allj = j+s # total cose with regularization

        # update weights
        for i in range(len(weights_list)):
            weights_list[i] -= learning_rate*gradients_batch[i]

    return weights_list, allj, j

In [65]:
normalizeinuse = n_ohe_wine_data
categoryinuse = ohe_wine_category
layerdataparameter = [5,4,3]

In [78]:
initweight = initialize_weights(categoryinuse,layerdataparameter)
updated_weight, jsum, purej = neural_network(normalizeinuse,categoryinuse,initweight)

In [66]:
def train_neural_network(normed_ohetraining_data,ohe_category,layerparameter, minibatchk = 15, lambda_reg = 0.15, learning_rate = 0.01, epsilon_0 = 0.00001, softstop = 8000, printq = False):
    init_weight = initialize_weights(ohe_category,layerparameter)
    updated_weight, jsum, purej = neural_network(normed_ohetraining_data,ohe_category,init_weight, minibatchk, lambda_reg, learning_rate)
    epsilon = epsilon_0 + 20
    currentj = jsum
    smallestj = jsum
    count = 0
    jlist = []
    jlist.append(currentj)
    while ((epsilon > epsilon_0) or (count < softstop) or (currentj >= smallestj)) and (count < (softstop+5000)):
        if printq:
            print('currentj',currentj)
            print('count',count)
        count += 1
        updated_weight, jsum, purej = neural_network(normed_ohetraining_data,ohe_category,updated_weight,minibatchk,lambda_reg,learning_rate)
        epsilon = jsum - currentj
        currentj = jsum
        jlist.append(currentj)
        if currentj < smallestj:
            smallestj = currentj
    
    return updated_weight, jlist

In [113]:
def predictoneinstance(inputdata,weightl):
    current_layer_a = np.append(1,inputdata)
    current_layer_index = 0
    alist = []
    alist.append(current_layer_a)
    for theta in weightl:
        z = np.dot(theta,current_layer_a)
        a = g(z)
        current_layer_a = np.append(1,a) if (current_layer_index+1 != len(weightl)) else a
        alist.append(current_layer_a)
        current_layer_index += 1
        raw_output = a
    predict_output = current_layer_a

    if len(predict_output) <=1:
        predict_output[0] = 0 if predict_output[0] <= 0.5 else 1
    else:
        predict_output[np.where(predict_output==np.max(predict_output))] = 1
        predict_output[np.where(predict_output!=1)] = 0
    
    return predict_output, raw_output 

In [ ]:
def predict_many_nn(testdatafull, ohecategory, weight):
    n = 0 
    inputindex, outputindex = [],[]
    for i in ohecategory:
        if ohecategory[i] != 'class_numerical':
            inputindex.append(n)
        else:
            outputindex.append(n)
        n += 1
    predictvsexpectlist = [] # list of list of predict and expect/actual

    for instance in testdatafull:
        datainput = instance[inputindex]
        expect_output = instance[outputindex]
        predict_output, raw_output = predictoneinstance(datainput,weight)
        # process the index of value 1 in np.array
        processdexpect = np.where(expect_output==1)[0][0]
        processdpredict = np.where(predict_output==1)[0][0]
        predictvsexpectlist.append([processdpredict,processdexpect])

    correct = 0
    for outputtup in predictvsexpectlist:
        if outputtup[0] == outputtup[1]:
            correct += 1
    accuracy = correct/len(predictvsexpectlist)

    return predictvsexpectlist, accuracy
    